In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

inputs = tf.keras.layers.Input(shape=(5,))
hidden = tf.keras.layers.Dense(2000, activation="relu", name="hidden")
outputs = tf.keras.layers.Dense(1)(hidden(inputs))
model = tf.keras.models.Model(inputs = inputs, outputs = outputs)


In [2]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
hidden (Dense)               (None, 2000)              12000     
_________________________________________________________________
dense (Dense)                (None, 1)                 2001      
Total params: 14,001
Trainable params: 14,001
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer='sgd', loss = 'mse')
model.metrics_names

[]

In [4]:
import numpy as np
x1, x2 = np.random.normal(size = (200, 5)), np.random.normal(size = (20, 5))
beta, delta = np.array([1, 1, 0, 0, 0]).reshape((-1,1)), np.array([0, 0, 0.2, 0.2, 0.2]).reshape((-1,1))
y1, y2 = x1 @ beta + 0.1 * np.random.normal(size=(200, 1)), x2 @ (beta + delta) + 0.1 * np.random.normal(size = (20, 1))
x, y = np.vstack((x1, x2)), np.vstack((y1, y2))

In [5]:
x_test = np.random.normal(size = (1000, 5))
y_test = x_test @ (beta + delta) + 0.1 * np.random.normal(size = (1000, 1))

In [8]:
model.fit(x, y, epochs = 200)

Epoch 1/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0175 - mse: 0.0175
Epoch 2/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0175 - mse: 0.0175
Epoch 3/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0174 - mse: 0.0174
Epoch 4/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0174 - mse: 0.0174
Epoch 5/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0174 - mse: 0.0174
Epoch 6/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0173 - mse: 0.0173
Epoch 7/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0174 - mse: 0.0174
Epoch 8/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0173 - mse: 0.0173
Epoch 9/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0173 - mse: 0.0173
Epoch 10/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0173 - mse: 0.0173
Epoch 11/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0172 

In [7]:
model.evaluate(x_test, y_test)

32/32 [==============================] - 0s 726us/step - loss: 0.1111 - mse: 0.1111


[0.11109097301959991, 0.11109097301959991]